In [4]:
import numpy as np
import vtrigU as vtrig
import matplotlib.pyplot as plt
from scipy import signal
from numpy.linalg import norm
from math import ceil, log
from scipy import constants
from vtrigU_helper_functions import *

# define some constants
c = constants.c
ant_loc = ants_locations()
ant_loc = ant_loc[20:,:2]
ant_array = np.dstack((ant_loc[10:,:],ant_loc[:10,:]))

In [6]:
# initialize the device
vtrig.Init()

# set setting structure
vtrigSettings = vtrig.RecordingSettings(
        vtrig.FrequencyRange(62.0*1000, # Start Frequency (in MHz)
                             66.5*1000, # Stop  Frequency (in MHz) (66.5 for 5m)
                             150),      # Number of Frequency Points (Maximum: 150)
        510.0,                           # RBW (in KHz)
        vtrig.VTRIG_U_TXMODE__LOW_RATE  # Tx Mode (LOW: 20 Tx, MED: 10 Tx, HIGH: 4 Tx)
        ) 

# validate settings
vtrig.ValidateSettings(vtrigSettings)

# apply settings
vtrig.ApplySettings(vtrigSettings)

# get antenna pairs and convert to numpy matrix
TxRxPairs = np.array(vtrig.GetAntennaPairs(vtrigSettings.mode))

# get used frequencies in Hz
freq = np.array(vtrig.GetFreqVector_MHz()) * 1e6

# define constants
N_txrx = TxRxPairs.shape[0]
N_freq = freq.shape[0]

Nfft = 2**(ceil(log(freq.shape[0],2))+1)
Ts = 1/Nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
time_vec = np.linspace(0,Ts*(Nfft-1),num=Nfft)
dist_vec = time_vec*(c/2) # distance in meters

vtrigUError: Init: The SDK is unable to communicate with the instrument

In [5]:
# Record Data
nframes = 100
data = []
for frame in range(nframes):
    vtrig.Record()
    rec = vtrig.GetRecordingResult()
    recArr = rec2arr(rec)[:20,0]
    data.append(recArr)
data = np.array(recArr)
print(data.shape)

NameError: name '_vtrig' is not defined

In [22]:
def path_difference(ant_loc,theta):
    reference_distance = 10000
    reference_ant_loc = ant_loc[0]
    target_loc = reference_ant_loc + np.array([reference_distance*np.cos(theta),reference_distance*np.sin(theta)])
    mic_distance = []
    for idx in range(len(ant_loc)-1):
        cur_loc = ant_loc[idx+1]
        cur_dis = norm(target_loc,cur_loc)
        mic_distance.append(cur_dis - reference_distance)
    mic_distance = np.array(mic_distance)
    return mic_distance

In [21]:
def aoa_localization(ant_loc, FS, data):
    """AoA localization algorithm. Write your code here.

    Args:
        mic_data_folder: name of folder (without a trailing slash) containing 
                         the mic datafiles `0.csv` through `7.csv` and `config.csv`.
        FS: microphone sampling frequency - 16kHz.
        MIC_OFFSETS: a list of tuples of each microphone's location relative to the center of its mic array. 
                     This list is calculated based on the diameter(0.09218m) and geometry of the microphone array.
                     For example, MIC_OFFSETS[1] is [0.09218*0.5, 0]. If the location of microphone array #i is
                     [x_i, y_i]. Then [x_i, y_i] + MIC_OFFSETS[j] yields the absolute location of mic#j of array#i.
                     This is provided for you r convenience and you may choose to ignore.

    Returns:
        The user's location in this format (in metres): (L_x, L_y)

    """
    
    # Calculate AoA Spectrum
    aoa = []
    angle_list = np.linspace(-180,180,360) * np.pi / 180
    aoa_spectrums = {}
    #print(mic_data[1,:,:])
    # Loop for all the mic arrays
    for array in range(2):
        aoa_spectrums[array] = {}
        aoa_correlations = []
        # Loop from -pi to pi
        for i in np.linspace(-180,180,360):
            cur_arr = np.copy(data[array,:,:])
            #print(cur_arr.shape)
            theta = i * np.pi / 180
            reference_signal = cur_arr[:,0]
            mic_distance = path_difference(array_locs[mic_array],mic_offsets,theta)
            shifted = []
            shifted.append(reference_signal)
            # Perform time shift
            for j in range(5):
                #print(cur_arr)
                time_shift = round((mic_distance[j] / sound_speed) * FS)
                #print(mic_data[mic_array,:,j])
                #print(time_shift)
                #print(cur_arr[:,j+1])
                if time_shift < 0:   
                    cur_shifted = np.concatenate((np.zeros(np.abs(time_shift)),cur_arr[:time_shift,j+1]))
                elif time_shift > 0:
                    cur_shifted = np.concatenate((cur_arr[time_shift:,j+1],np.zeros(np.abs(time_shift))))
                else:
                    cur_shifted = cur_arr[:,j+1]
                #print(cur_shifted)
                #print(' ')
                shifted.append(cur_shifted)
            shifted = np.array(shifted).T
            #print(shifted)
            correlation = 0
            for k in range(5):
                correlation = correlation + signal.correlate(reference_signal,shifted[:,k+1],mode="valid",method="direct")
            #print(shifted.shape)
            #error.append(eu_dis(reference_signal,dns_data)) 
            #print(signal.correlate(dns_data,reference_signal,mode="valid"))
            aoa_correlations.append(correlation)
        aoa_correlations = np.array(aoa_correlations)
        
        aoa.append(angle_list[np.argwhere(aoa_correlations == np.max(aoa_correlations))[:,0].flatten()[-1]])
        # Normalization
        aoa_correlations = aoa_correlations/np.linalg.norm(aoa_correlations)
        
        for n in range(len(aoa_correlations)):
            aoa_spectrums[mic_array][angle_list[n]] = aoa_correlations[n]
        
        #print(np.max(error),np.argmax(error))
        #print(np.max(error))
        #print(error[47])
        #print(np.argwhere(error == np.max(error))[:,0].flatten())
        
        
        plt.figure()
        print(np.linspace(-180,180,360)[np.argmax(aoa_correlations)])
        plt.plot(np.linspace(-180,180,360),aoa_correlations)
        plt.show()

    # Find Borderlines
    MinX = 10000
    MinY = 10000
    MaxX = -10000
    MaxY = -10000
    for loc in array_locs:
        MinX = min(MinX, loc[0])
        MinY = min(MinY, loc[1])
        MaxX = max(MaxX, loc[0])
        MaxY = max(MaxY, loc[1])
    
    x_range = np.linspace(MinX,MaxX,100)
    y_range = np.linspace(MinY,MaxY,100)
    
    position_likelihood = {}
    for x in x_range:
        for y in y_range:
            likelihood = 1
            for i,array in enumerate(array_locs):
                if x == array[0] and y == array[1]:
                    likelihood = 0
                else:
                    cur_theta = np.arctan2((y-array[1]),(x-array[0]))
                    cur_theta = angle_list[np.argmin(np.abs(angle_list - cur_theta))]
                    likelihood = likelihood * aoa_spectrums[i][cur_theta]
            position_likelihood[(x,y)] = likelihood
    
    userLoc = max(position_likelihood, key=position_likelihood.get)
    
    # Your return value should be the user's location in this format (in metres): (L_x, L_y)
    
    return (userLoc[0], userLoc[1])

[[ 0.0274 -0.0133]
 [ 0.0274 -0.0112]
 [ 0.0274 -0.0091]
 [ 0.0274 -0.007 ]
 [ 0.0274 -0.0049]
 [ 0.0274 -0.0028]
 [ 0.0274 -0.0007]
 [ 0.0275  0.0014]
 [ 0.0275  0.0035]
 [ 0.0275  0.0056]
 [ 0.0275  0.0078]
 [ 0.0275  0.0099]
 [ 0.0275  0.012 ]
 [ 0.0274  0.0141]
 [ 0.0274  0.0162]
 [ 0.0275  0.0183]
 [ 0.0275  0.0204]
 [ 0.0275  0.0225]
 [ 0.0275  0.0246]
 [ 0.0275  0.0267]]
[ 0.0274 -0.0133]
